# Cleaning Censorship Data

## Requirements
First, make sure the requirements are installed. 

In [1]:
import seaborn as sns
import plotly.io as pio
import matplotlib.pyplot as plt
import plotly.express as px
import os
import pandas as pd
import sys
sys.path.append('../')

from IPython.core.getipython import get_ipython

pio.renderers.default = "png"

## Acquiring Data, Combining, and De-duplicating It

The code below combines eight total files with ETH L1 data into a single DataFrame. 

In [2]:
PARQUET_FILES_TO_USE = [
    "../data/data_000000000000.parquet",
    "../data/data_000000000001.parquet",
    "../data/data_000000000002.parquet",
    "../data/data_000000000003.parquet",
    "../data/data_000000000004.parquet",
    "../data/data_000000000005.parquet",
    "../data/data_000000000006.parquet",
    "../data/openethdata_eth_data.parquet__1_.gzip"
]

data = [pd.read_parquet(file) 
       for file 
       in PARQUET_FILES_TO_USE]

In [3]:
concatenated_data = pd.concat(data)

In the next cells, we drop duplicate rows, and make sure that the resulting data actually contains no duplicates. 

In [4]:
concatenated_data.drop_duplicates(inplace = True)

In [5]:
assert concatenated_data.duplicated().sum() == 0, "The concatenated_data frame should have no duplicate rows."

Next we filter the `concatenated_data` down to only the columns that are relevant to us:
* `date`
* `slot`
* `block_number`
* `builder`
* `validator`
* `builder_pubkey`
* `mevboost_value` 

In [6]:
filtered_concatenated_data = concatenated_data[['date',
                                                'slot',
                                                'block_number',
                                                'builder',
                                                'validator',
                                                'builder_pubkey',
                                                'mevboost_value']]

Now we check to see which columns in `filtered_concatenated_data` have missing values. 

In [7]:
filtered_concatenated_data.isnull().sum()

date                    0
slot                    0
block_number            8
builder            588427
validator               0
builder_pubkey     588427
mevboost_value    5355742
dtype: int64

We see that the number of entries in `builder` and `builder_pubkey` with missing values is the same. There are more `mevboost_value` entries that are missing. 

We investigate the conjecture that every row with `builder` and `builder_pubkey` missing also has `mevboost_value` missing. 

In [8]:
filtered_concatenated_data[
                           (filtered_concatenated_data['builder'].isna())
                           & (filtered_concatenated_data['builder_pubkey'].isna())
                           ]['mevboost_value'].isna().sum()

588427

These correspond to so-called **vanilla blocks**, where the builder and validator are the same. 
The function `update_builder` replaces the missing `builder` entries with the `validator`. 

In [9]:
def update_builder(df):
    mask = df['builder'].isnull() & df['builder_pubkey'].isnull() & df['mevboost_value'].isnull()
    df.loc[mask, 'builder'] = df.loc[mask, 'validator']
    return df

In [10]:
filled_in_data = update_builder(filtered_concatenated_data)

In [11]:
assert len(filled_in_data) == len(filtered_concatenated_data)

Now that this processing is done, we filter out the  `mevboost_value` column so there are no missing values left in the data. We check to see if there are any further missing values. 

In [12]:
filled_in_data = filled_in_data[['date', 'slot', 'block_number', 'builder', 'validator']]

In [13]:
filled_in_data.isna().sum() > 0

date            False
slot            False
block_number     True
builder         False
validator       False
dtype: bool

The `block_number` column still has missing entries. Let's check how many, and see where they are. 

In [14]:
filled_in_data['block_number'].isna().sum()

8

In [15]:
filled_in_data[filled_in_data['block_number'].isna()]

,date,slot,block_number,builder,validator
194300,2023-01-27 06:05:11,5664624,NaN,0xb9f106d153a8e96cbfd5cff0ef32e412159b03d5fad1...,0xb9f106d153a8e96cbfd5cff0ef32e412159b03d5fad1...
194301,2023-01-27 06:05:23,5664625,NaN,0xabbe54e11b0a4ac44794e9ca1bdf2eb0aca86ff5ac60...,0xabbe54e11b0a4ac44794e9ca1bdf2eb0aca86ff5ac60...
194307,2023-01-27 06:06:35,5664631,NaN,0xafd0dcfd03b1014f18af4ea70033700ec1c7e823cff3...,0xafd0dcfd03b1014f18af4ea70033700ec1c7e823cff3...
194311,2023-01-27 06:07:23,5664635,NaN,missed,missed
194324,2023-01-27 06:09:35,5664646,NaN,daniel wang,daniel wang
194327,2023-01-27 06:10:11,5664649,NaN,0xb6059f91f339534bddf303a19e7c8578b88ead002aba...,0xb6059f91f339534bddf303a19e7c8578b88ead002aba...
194331,2023-01-27 06:10:59,5664653,NaN,0xadf148cd30edd5bebf490cd59cdd84e0823389e62cd2...,0xadf148cd30edd5bebf490cd59cdd84e0823389e62cd2...
216471,2023-01-27 06:09:35,5664646,NaN,0x8a0a68a2b4a4b50dada2dc79ecefdff79ac39fee62f4...,0x8a0a68a2b4a4b50dada2dc79ecefdff79ac39fee62f4...


In [16]:
len(filled_in_data)

8505048

Since the entries are from early 2023, they are unlikely to be relevant to our data. We drop them all. 

In [17]:
filled_in_data.dropna(axis = 'index',
                    inplace = True)

C:\Users\andre\AppData\Local\Temp\ipykernel_18440\421032421.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
len(filled_in_data)

8505040

In [19]:
assert filled_in_data.isna().sum().sum() == 0, "We are seeing missing values where we would not expect them. "

## Sorting Data and Checking Information

We determine the type of information used in the `date` column, and use it to sort the DataFrame. 

In [20]:
type(filled_in_data['date'].iloc[0])

pandas._libs.tslibs.timestamps.Timestamp

In [21]:
sorted_data = filled_in_data.sort_values(by='date')

In [22]:
assert sorted_data.isna().sum().sum() == 0, "This DataFrame should have no missing values. "

In [23]:
sorted_data.isna().sum()

date            0
slot            0
block_number    0
builder         0
validator       0
dtype: int64

In [24]:
sorted_data['slot'].is_monotonic_increasing

True

In [25]:
sorted_data['block_number'].is_monotonic_increasing

False

In [26]:
sorted_data[sorted_data['block_number'].diff() < 0]

,date,slot,block_number,builder,validator
177153,2022-09-15 06:53:23,4700065,0.0,missed,missed
3338372,2022-09-15 07:02:59,4700113,0.0,missed,missed
282694,2022-09-15 07:17:35,4700186,0.0,missed,missed
3338724,2022-09-15 07:23:59,4700218,0.0,missed,missed
2181084,2022-09-15 08:13:35,4700466,0.0,missed,missed
...,...,...,...,...,...
613794,2024-04-24 03:22:47,8925412,0.0,missed,missed
614289,2024-04-24 04:16:47,8925682,0.0,missed,missed
614990,2024-04-24 05:31:11,8926054,0.0,missed,missed
614996,2024-04-24 05:31:59,8926058,0.0,missed,missed


This raises the issue that the data source is using `block_number = 0.0` with `builder = missed` and `validator = missed` as an alternative way of indicating missed blocks, in addition to `None` or `NaN`. 

We calculate the proportion using this encoding, since they could be dropped. 

In [27]:
len(sorted_data[sorted_data['block_number'] == 0])/len(sorted_data)

0.0038304346599192947

Since this data accounts for less than 0.4% of the total data, we can only consider block numbers that are positive. 

In [28]:
sorted_data = sorted_data[sorted_data['block_number'] > 0]

Next, we compare the length of the `sorted_data` DataFrame to the number of unique values for `block_number` and `slot`. 

In [29]:
len(sorted_data)

8472462

In [30]:
len(sorted_data['block_number'].unique())

4185174

In [31]:
len(sorted_data['slot'].unique())

4195967

The number of `slot`s is slightly larger than the `block_number`, but this is expected behavior. We do a bit more sanity checking here, to see which slots and block numbers are covered by the data. 

In [32]:
min(sorted_data['block_number'])

15537394.0

In [33]:
max(sorted_data['block_number'])

19723375.0

In [34]:
min(sorted_data['slot'])

4700013

In [35]:
max(sorted_data['slot'])

8926261

## Removing Duplicate Rows, Plus Duplicate Entries in Slot and Block Number

In [39]:
sorted_df = sorted_data.sort_values(by="date")
dencun_df = sorted_df[(sorted_df["block_number"] > 19426589)]

In [40]:
dencun_df.drop_duplicates(inplace = True)

C:\Users\andre\AppData\Local\Temp\ipykernel_18440\453479485.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [41]:
assert dencun_df.duplicated().sum() == 0, "This DataFrame should have no repeated values. "

In [42]:
dencun_df.drop_duplicates(subset=['slot'], inplace=True)
dencun_df.drop_duplicates(subset=['block_number'], inplace=True)

C:\Users\andre\AppData\Local\Temp\ipykernel_18440\3843717115.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\andre\AppData\Local\Temp\ipykernel_18440\3843717115.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [43]:
assert dencun_df['slot'].duplicated().sum() == 0, "There are unexpected duplicate slot entries in the data."
assert dencun_df['block_number'].duplicated().sum() == 0, "There are unexpected duplicate block number entries in the data."

In [44]:

# Checking whether rows, unique blocks and unique slots are equal
unique_blocks = dencun_df['block_number'].nunique()
unique_slots = dencun_df['slot'].nunique()

total_rows = len(dencun_df)

same_number_of_uniques = unique_blocks == unique_slots

uniques_match_total = unique_blocks == total_rows and unique_slots == total_rows

print(f"Number of unique 'block_number' values: {unique_blocks}")
print(f"Number of unique 'slots' values: {unique_slots}")
print(f"Total number of rows in the DataFrame: {total_rows}")
print(
    f"Same number of unique values in both columns: {same_number_of_uniques}")
print(f"Number of unique values matches total rows: {uniques_match_total}")


Number of unique 'block_number' values: 296786
Number of unique 'slots' values: 296786
Total number of rows in the DataFrame: 296786
Same number of unique values in both columns: True
Number of unique values matches total rows: True


## Finishing by Saving the Data

At this point, we feel good about calling the data final and saving it. 

In [47]:
dencun_df.to_csv("../data/auxiliary/eth_builder_validator_data.csv.gz",
                          compression = "gzip")

In [48]:
dencun_df.to_parquet("../data/auxiliary/eth_builder_validator_data.parquet.gz",
                          compression = "gzip")

In [49]:
dencun_data_from_csv = pd.read_csv("../data/auxiliary/eth_builder_validator_data.csv.gz")

In [52]:
    assert dencun_data_from_csv.isna().sum().sum() == 0, "The data should have no missing values."
    assert dencun_data_from_csv.duplicated().sum() == 0, "The data should have no missing values."
    assert dencun_data_from_csv['slot'].duplicated().sum() == 0, "There are unexpected duplicate slot entries in the data."
    assert dencun_data_from_csv['block_number'].duplicated().sum() == 0, "There are unexpected duplicate block number entries in the data."
    assert len(dencun_data_from_csv) == dencun_data_from_csv['slot'].nunique(), "Number of slots should be the same as number of entries in data."
    assert (dencun_data_from_csv['slot'].nunique()) == (dencun_data_from_csv['block_number'].nunique()), "Number of slots should be the same as number of blocks in data."


In [54]:
dencun_data_from_parquet = pd.read_parquet("../data/auxiliary/eth_builder_validator_data.parquet.gz")

In [56]:
    assert dencun_data_from_parquet.isna().sum().sum() == 0, "The data should have no missing values."
    assert dencun_data_from_parquet.duplicated().sum() == 0, "The data should have no missing values."
    assert dencun_data_from_parquet['slot'].duplicated().sum() == 0, "There are unexpected duplicate slot entries in the data."
    assert dencun_data_from_parquet['block_number'].duplicated().sum() == 0, "There are unexpected duplicate block number entries in the data."
    assert len(dencun_data_from_parquet) == dencun_data_from_parquet['slot'].nunique(), "Number of slots should be the same as number of entries in data."
    assert (dencun_data_from_parquet['slot'].nunique()) == (dencun_data_from_parquet['block_number'].nunique()), "Number of slots should be the same as number of blocks in data."
